# Data exploration

I have exported my data from Polar watch. Exploring what is there 

In [34]:
import glob
import os
import json
import random

Define consts

In [12]:
DATA_DIR = "/Users/arbatov/gitRepo/gpx-data/data"

## Get the data

Load the files

In [13]:
json_files = glob.glob(os.path.join(DATA_DIR,"*.json"))

How many files?

In [14]:
print(len(json_files))

815


How big are all of the files?

In [18]:
nbytes = sum(d.stat().st_size for d in os.scandir(DATA_DIR) if d.is_file())
print(float(nbytes) / 1024**3, "GB")

1.7585922731086612 GB


## Overview of data

What kind of activities are there?

In [39]:
activities = {}

sampled_json_files = random.sample(json_files, 200)
for json_file in sampled_json_files:
    with open(json_file) as file:
        contents = json.load(file)
        activity = contents['exercises'][0]['sport']
        if activity in activities:
            activities[activity] += 1
        else:
            activities[activity] = 1

sorted_activities = dict(sorted(
    activities.items(), 
    key=lambda activity: activity[1],
    reverse=True
))            
for activity, count in sorted_activities.items():
    print(activity, count)

RUNNING 106
OPEN_WATER_SWIMMING 26
STRETCHING 20
STRENGTH_TRAINING 18
CYCLING 9
POOL_SWIMMING 7
TREADMILL_RUNNING 5
INDOOR_CYCLING 4
YOGA 3
WALKING 1
ROAD_RUNNING 1
